In [1]:
import pandas as pd
import numpy as np
import configparser
import sys
from sklearn.decomposition import PCA
config = configparser.ConfigParser()
config.read("../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
local_lib = config['DEFAULT']['local_lib']
sys.path.append(local_lib)

OHCO = ['speech_id', 'para_num', 'sent_num', 'token_num']

In [5]:
LIB = pd.read_csv('LIB.csv', sep='|').set_index('speech_id')
VOCAB = pd.read_csv('VOCAB.csv', sep='|').set_index('term_str')
TFIDF = pd.read_csv('TFIDF_L2.csv').set_index(['speech_id', 'para_num'])

In [7]:
n_comps = 10
pc_cols = [f"PC{i}" for i in range(n_comps)]
pca_engine = PCA(n_components=n_comps)
DCM = pd.DataFrame(pca_engine.fit_transform(TFIDF.fillna(0)), index=TFIDF.index).reset_index().set_index(
    ['speech_id', 'para_num'])
DCM['speech_id'] = DCM['speech_id'] + 1
DCM.columns = pc_cols
#print(DCM)
DCM = DCM.join(LIB, on='speech_id')
LOADINGS = pd.DataFrame(pca_engine.components_.T * np.sqrt(pca_engine.explained_variance_))
LOADINGS.columns = ["PC{}".format(i) for i in LOADINGS.columns]
LOADINGS.index = TFIDF.columns
LOADINGS.index.name = 'term_str'
LOADINGS = LOADINGS.join(VOCAB)

KeyError: 'speech_id'

In [ ]:
data = []
for i in range(n_comps):
    for j in [0,1]:
        data.append((f"PC{i}", j, ' '.join(LOADINGS.sort_values(f'PC{i}', ascending=bool(j)).head(10).index.to_list())))
comp_strs = pd.DataFrame(data)
comp_strs.columns = ['pc', 'pole', 'top_terms']
comp_strs = comp_strs.set_index(['pc', 'pole'])

In [ ]:
comp_strs.to_csv('components.csv')
comp_strs

In [ ]:
DCM.to_csv('DCM.csv')
DCM

In [ ]:
LOADINGS.to_csv('LOADINGS.csv')
LOADINGS